In [3]:
from __future__ import print_function, division

import os
import os.path
import pandas as pd
from io import StringIO
import io

import numpy as np
from numpy import array

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [16]:
SEQUENCE_LENGTH = 40
SEQUENCE_LENGTH_D = 25
#read train and val data
fpath = 'data/TOEFL'

In [17]:
SEQUENCE_LEN_D = SEQUENCE_LENGTH_D
SEQUENCE_LEN = SEQUENCE_LENGTH

df_train = pd.read_csv(os.path.join(fpath, 'train.csv'))
text = df_train['text1']
df_val = pd.read_csv(os.path.join(fpath,'test.csv'))

text_val = df_val['text1']
text_train = df_train['text1']
rank_val = df_val['label']
rank_train = df_train['label']

target_val = np.array(rank_val)
target_train = np.array(rank_train)

onehot_encoder = OneHotEncoder(sparse=False)

integer_encoded = target_train.reshape(len(target_train), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)

integer_encoded_val = target_val.reshape(len(target_val), 1)
y_test = onehot_encoder.fit_transform(integer_encoded_val)


In [18]:
### Get bert senence mebeddings for each sentence in an essay
## Based on BERT serving client https://pypi.org/project/bert-serving-client/
## base uncased model
from nltk import sent_tokenize
from bert_serving.client import BertClient
bc = BertClient()
b_len = 768 

/anaconda3/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=40" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [20]:
X_train = []

for i in df_train['text1']:
    i = sent_tokenize(i)
    X_train.extend(bc.encode(i[:SEQUENCE_LEN_D]))
    for k in range(max(SEQUENCE_LEN_D - (len(i)), 0)):
        X_train.append([[0]*b_len]*SEQUENCE_LEN) # pad token maps to 0
    #break
    
X_train = np.array(X_train)  

np.save('X_train_TOEFL', X_train)
np.save('y_train_TOEFL', y_train)

In [21]:
del X_train
del y_train
import gc
gc.collect()

0

In [22]:
#val set
X_test = []

for i in df_val['text1']:
    i = sent_tokenize(i)
    X_test.extend(bc.encode(i[:SEQUENCE_LEN_D]))
    for k in range(max(SEQUENCE_LEN_D - (len(i)), 0)):
        X_test.append([[0]*b_len]*SEQUENCE_LEN) # pad token maps to 0
X_test = np.array(X_test)


/anaconda3/lib/python3.6/site-packages/bert_serving/client/__init__.py:286: UserWarning: some of your sentences have more tokens than "max_seq_len=40" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [23]:
#X_test.shape[0]/SEQUENCE_LEN_D
np.save('X_test_TOEFL', X_test)
np.save('y_test_TOEFL', y_test)